In [ ]:
%load_ext snakeviz
%load_ext autoreload
%autoreload 2
from scipy.io import wavfile
import numpy as np
import matplotlib.pyplot as plt
import IPython.display as ipd
import librosa
import librosa.display
from scipy import sparse
from letitbee import *
from particle import *
from probutils import *
import time

# Let It Bee

In [ ]:
win = 2048
hop = 1024
p = 10

y1, sr = librosa.load("Beatles_LetItBe.mp3")
y2, sr = librosa.load("Bees_Buzzing.mp3")

W1 = get_windowed(y1, hop, win)
V = np.abs(np.fft.fft(W1, axis=0)[0:win//2+1, :])
WSound = get_windowed(y2, hop, win)
W = np.abs(np.fft.fft(WSound, axis=0)[0:win//2+1, :])

In [ ]:
HGT = get_musaic_activations(V, W, L=50, r=3, p=p, c=3, verbose=False)
WH = W.dot(HGT)
print(np.sum(V*np.log(V/WH) - V + WH))
y = do_windowed_sum(WSound, HGT, win, hop)
ipd.Audio(y, rate=sr)

In [ ]:
pd = 0.95
sigma = 0.25
L = 10
P = 1000
neff_thresh = 100
tic = time.time()
H, wsmax, neff = get_particle_musaic_activations(V, W, p, pd, sigma, L, P, gamma=0, 
                                                 neff_thresh=neff_thresh, use_gpu=False)
print("Elapsed time musaic: {:.3f}".format(time.time()-tic))

a1 = get_activations_diff(HGT, p)
a2 = get_activations_diff(H, p)

WH = W.dot(H)
fit = np.sum(V*np.log(V/WH) - V + WH)
y = do_windowed_sum(WSound, H, win, hop)
ipd.Audio(y, rate=sr)

In [ ]:
y = y/np.max(y)
prefix = "p{}_pd{}_sigma{}_L{}_P{}_neff{}".format(p, pd, sigma, L, P, neff_thresh)
wavfile.write("{}.wav".format(prefix), sr, y)

plt.figure(figsize=(20, 4))
plt.subplot(141)
plt.plot(a1)
plt.plot(a2)
plt.legend(["Ground Truth: Mean {:.3f}".format(np.mean(a1)),
            "Particle Filter: Mean {:.3f}".format(np.mean(a2))])
plt.subplot(142)
plt.hist(a1, bins=np.arange(p+2), alpha=0.5)
plt.hist(a2, bins=np.arange(p+2), alpha=0.5)
plt.subplot(143)
plt.plot(wsmax)
plt.title("Max Probability, Overall Fit: {:.3f}".format(fit))
plt.xlabel("Timestep")
plt.subplot(144)
plt.plot(neff)
plt.xlabel("Timestep")
plt.title("Number of Effective Particles")
plt.savefig("{}.svg".format(prefix), bbox_inches='tight')


